In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import re
import time
import bs4 as bs4
import requests as rq
import json
import os 
import tqdm
import glob

pd.set_option('max_colwidth', None)

In [2]:
car_features = ['brand', 'price','cartype','model','gearbox',
            'regdate', 'mileage', 'motorpower', 'fuel', 
            'car_steering', 'carcolor',  'exchange','version','doors','financial','extra']

In [3]:
def is_null_test(feat_):
    '''Verifies whether a variable is Null
    ARG:
    feat_(re): A re object containing the car information
    RETURNS:
    True(bool): A bool variable stating whether the object is null or not'''
    
    
    is_null = 'null' in feat_.group(0)
    if is_null == True:
        return True

In [4]:
def is_empty_test(car_info):
    
    '''Tests whether the car_info object is empty or not
    ARG: 
    car_info(bs4 object) : A beautifulsoup object containing car information
    
    RETURNS:
    True (bool): A bool variable stating whether the object is empty or not'''
    
    is_empty = not car_info
    
    if is_empty == True:
    
        return True
    

In [8]:
def car_info_parser(car_info, car_link):
    
    '''Gets car information and put it into a dictionary
    ARG:
    car_info(bs4 object):A beautifulsoup object containing car information
    car_link(string) : A string containing the link of the car
    
    RETURNS:
    data(dictionary): A dictionary containing the car features as key'''

    # features
    car_features = ['brand', 'price','cartype','model','gearbox',
                'regdate', 'mileage', 'motorpower', 'fuel', 
                'car_steering', 'carcolor',  'exchange','version','doors','financial','extra']


    data = {}
    
    # add car link to dict
    data['link'] = car_link 

    for feat in car_features:
        
        
        # Verify whether the feature is in the car_info[0] string
        contain_feature = feat in car_info[0]
        
        
        # Get all features information by using regular expressions
        if contain_feature == True:
            

            if feat in ['version','doors','financial']:

               
                feat_ = re.search("{variable}(([^\,]*))".format(variable = feat), car_info[0])
                
                # null test
                if is_null_test(feat_) == True:
                      data[feat] = 0.0
                else:        
                    feature = re.search(":\"(([^\,]*))",feat_.group(0))
                    text = re.sub(r"[^a-zA-Z0-9ãáéóõç.]", " ",  feature.group(0).lower()) 

                    data[feat] = text.strip()


            elif feat == 'extra':

                feat_ = re.search(".(?=extra)([^\}]*)", car_info[0])
                
                # Null test
                if is_null_test(feat_) == True:
                      data[feat] = 0.0
                    
                else:  
                    feature = re.search(":\"(([^\}]*))",feat_.group(0))
                    text = re.sub(r"[^a-zA-Z0-9ãáéóõç.,]", " ",  feature.group(0).lower())
                    
                    data[feat] = text.strip()


            else:      

                feat_ = re.search("{variable}(([^\,]*))".format(variable = feat), car_info[0]) 
                
                # Null test
                if is_null_test(feat_) == True:
                    
                    data[feat] = 0.0           
                else:
                    feature = re.search(":\"(([^\,]*))",feat_.group(0))
                    text = re.sub(r"[^a-zA-Z0-9ãáéóõç.]", "",  feature.group(0).lower()) 
                 
                    col = feat
                    data[col] = text.strip()

        # If feature is not in car_info[0] string, sets 0 as value
        elif contain_feature == False:
           
            data[feat] = 0.0
    
    
    return data
        

In [9]:
base_link = 'https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/'

In [10]:
with open("parsed_link_info.json", 'w+') as output:
    
    for link_file in tqdm.tqdm_notebook(sorted(glob.glob("./raw_data/links/link_*"))):
        
        # Car link identifier 
        link_id = re.search(r"(link_)([^\.]*)",link_file)
        link_id = link_id.group(0)
        
        #car link
        car_link = base_link + link_id
        print(base_link + link_id)
       
        
        with open(link_file,'r+') as inp:
            
            page_html = inp.read()
            
            parsed = bs4.BeautifulSoup(page_html, 'html.parser')
            
            # Find the piece of HTML where I will find the car information
            car_info = parsed.find_all(string=re.compile("window.dataLayer"))
            
            # test whether the bs4 obj(car_info)is empty
            if is_empty_test(car_info) == True:
                pass
            else:

                data = car_info_parser(car_info,car_link )

                output.write("{}\n".format(json.dumps(data)))


https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_-unica-dona-729128175
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_0vendo-gol-1-0-com-ar-condicionado-baixo-km-734865745
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_1-6-fiesta-2012-2013-730821135
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_13-13-completo-km-35-000-2020-vistoriado-731365923
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_16-16-unico-dono-muito-novo-713990866
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_2014-1-6-sedan-entrada-3mil-ipva-pago-completo-novo-729265900
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_2015-gol-volkswagen-726466377
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-u

https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_carro-716446857
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_carro-716795265
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_carro-717336180
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_carro-717635830
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_carro-718226325
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_carro-719007329
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_carro-719258117
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_carro-720969973
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_carro-722107147
https://rj.olx.com.br/rio-de-janeiro-

https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_carro-fox-2013-2014-1-0-carro-de-mulher-729130809
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_carro-fox-714790803
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_carro-fox-comfortline-735381604
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_carro-fox-trendline-msi-732995688
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_carro-fox-vendo-2013-2014-tel-leilao-636567938
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_carro-gol-715931400
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_carro-gol-confortline-737322253
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_carro-mto-novo-728040145
https://rj.o

https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_crossfox-1-6-2013-unico-dono-29-900-737782188
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_crossfox-1-6-2014-718011571
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_crossfox-1-6-2014-726208495
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_crossfox-1-6-2014-manual-2-dono-728653930
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_crossfox-1-6-727197254
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_crossfox-1-6-736916085
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_crossfox-1-6-ano-e-modelo-2016-727100928
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_crossfox-1-6-completo-sem-detalhe-7380454

https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_crossfox-unico-dono-ipva-2020-pago-apenas-16-km-rodados-completasso-novo-d--714031605
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_crossfox-unico-dono-pouquissimo-rodado-716590944
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_crossfox-vermelho-tornado-712387997
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_da-pandemia-a-solucao-729971968
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_documento-ok-724414105
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_duvido-igual-ecosport-1-6-freestyle-flex-4p-aut-640371352
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_duvido-igual-fiesta-1-6-titanium-hatch-16v-4p-aut-640372435
https://rj.olx.com.

...

https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-jetta-trendline-1-4-tsi-tiptronic-736932759
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-jetta-tsi-2014-entrada-48x-831-25-fixas-ipva-pago-738095441
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-jetta-tsi-731837404
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-kombi-687060847
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-kombi-furgao-1-4-8v-flex-724762222
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-kombi-last-edition-724592054
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-new-crossfox-1-6-imotion-2017-unico-dono-727936921
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/l

https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-saveiro-trend-c-s-2013-ger-5-flex-1-6-8v-732377094
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-saveiro-trendline-2016-completo-726368232
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-saveiro-trendline-ce-ano-2017-738108748
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-spacecross-1-6-8v-top-de-linha-ler-anuncio-695327457
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-spacecross-1-6-total-flex-2013-736422341
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-spacecross-2014-completa-727613338
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-spacefox-1-6-com-gnv-nova-troco-e-financio-aceito-carro-ou-moto-548046042
https:

https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-up-take-2016-completo-26-000-00-727583360
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-up-take-2018-731057260
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-up-take-736338666
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-up-take-completo-ipva-2020-pago-738062062
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-up-take-ma-ano-2016-completo-678346811
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-up-take-muito-novo-unico-dono-financiamos-sem-comprovacao-de-renda-709043004
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-up-tsi-1-0-2016-735749180
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-u

https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-volkswagen-saveiro-2013-ce-1-6-flex-gnv-629803255
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-volkswagen-saveiro-536695520
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-volkswagen-saveiro-572233278
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-volkswagen-saveiro-cross-ce-2013-723329235
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-volkswagen-saveiro-cs-1-6-completo-ano-2014-728523721
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-volkswagen-space-fox-comfortline-1-6-mi-total-flex-8v-5p-em-otimo-estado-729587874
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-volkswagen-up-2015-638318433
https://rj.olx.com.br/rio

https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-voyage-trendline-2015-em-otimo-estado-736557649
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_vw-voyage-trendline-2017-738423474
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_wolks-up-take-2015-723752441
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_wolksvagem-voyage-725195495
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_wv-fox-2013-obs-emplacado-2020-704238192
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_wv-saveiro-1-6-completa-baixa-km-financio-60x-fixas-712032451
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_wv-up-1-0-flex-2pts-727587862
https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/l

## DataFrame

In [11]:
df = pd.read_json('parsed_link_info.json',lines = True)
df.tail()

,link,brand,price,cartype,model,gearbox,regdate,mileage,motorpower,fuel,car_steering,carcolor,exchange,version,doors,financial,extra
6857,https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_wv-saveiro-1-6-completa-baixa-km-financio-60x-fixas-712032451,vwvolkswagen,33900,vanutilitário,saveiro,manual,2017,69000,1.6,flex,hidráulica,branco,sim,saveiro robust 1.6 total flex 8v,2 portas,ipva pago,"vidro elétrico, air bag, trava elétrica, ar condicionado, direção hidráulica, alarme, som, sensor de ré"
6858,https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_wv-up-1-0-flex-2pts-727587862,vwvolkswagen,26880,hatch,up,manual,2016,66,1.0,flex,hidráulica,preto,sim,up take 1.0 t. flex 12v 3p,4 portas,ipva pago,"vidro elétrico, air bag, trava elétrica, ar condicionado, direção hidráulica, alarme, som"
6859,https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_wv-up-2014-2015-737445508,vwvolkswagen,22500,hatch,up,manual,2014,53,1.0,flex,hidráulica,preto,não,up move i motion 1.0 t. flex 12v 3p,2 portas,ipva pago,"vidro elétrico, trava elétrica, ar condicionado, direção hidráulica, alarme, som"
6860,https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_wv-voyage-2016-confortiline-parcelas-fixas-de-r-615-00-714041737,vwvolkswagen,27900,0,voyage,0,2016,78321,0,flex,0,0,0,0,0,0,"vidro elétrico, air bag, trava elétrica, ar condicionado, direção hidráulica, alarme, som, sensor de ré"
6861,https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/link_wv-voyage-completo-trend-1-6-2018-715280723,vwvolkswagen,38900,sedã,voyage,manual,2018,36000,1.6,flex,hidráulica,preto,não,voyage trend 1.6 mi total flex 8v 4p,4 portas,ipva pago,"vidro elétrico, air bag, trava elétrica, ar condicionado, direção hidráulica, alarme, som, sensor de ré"
